In [9]:
#a: Handle Missing Values
import pandas as pd
import numpy as np
import os

# Load the dataset
df = pd.read_csv('/content/train.csv')  # Adjust path if running locally or differently

# Check for missing values
print("Missing values in each column:\n", df.isnull().sum())

# Preprocess to remove units and convert to numeric with error handling
# Handle 'Mileage' (remove 'kmpl' or 'km/kg')
df['Mileage'] = pd.to_numeric(df['Mileage'].str.extract(r'(\d+\.?\d*)')[0], errors='coerce')
# Handle 'Engine' (remove 'CC')
df['Engine'] = pd.to_numeric(df['Engine'].str.extract(r'(\d+\.?\d*)')[0], errors='coerce')
# Handle 'Power' (remove 'bhp')
df['Power'] = pd.to_numeric(df['Power'].str.extract(r'(\d+\.?\d*)')[0], errors='coerce')
# Handle 'New_Price' (remove 'Lakh')
df['New_Price'] = pd.to_numeric(df['New_Price'].str.extract(r'(\d+\.?\d*)')[0], errors='coerce')

# Impute numerical columns with median (avoiding inplace warning)
numeric_cols = ['Mileage', 'Engine', 'Power', 'Seats', 'New_Price']
for col in numeric_cols:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median())  # Assign back instead of inplace

# Drop rows where 'Price' is missing (target variable)
df.dropna(subset=['Price'], inplace=True)

# Verify no more missing values
print("Missing values after handling:\n", df.isnull().sum())

# Create directory if it doesn't exist
output_dir = 'Cars_Project_Assignment-2/Data_Clean'
os.makedirs(output_dir, exist_ok=True)

# Save the cleaned data
df.to_csv(os.path.join(output_dir, 'data_cleaned_missingValues.csv'), index=False)

Missing values in each column:
 Unnamed: 0              0
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  38
New_Price            5032
Price                   0
dtype: int64
Missing values after handling:
 Unnamed: 0           0
Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
New_Price            0
Price                0
dtype: int64


In [11]:
#b: Remove Units from Attributes
import pandas as pd
import os

# Load the dataset from the previous step (Part a)
df = pd.read_csv('Cars_Project_Assignment-2/Data_Clean/data_cleaned_missingValues.csv')

# Ensure columns are numeric and remove any residual units (safeguard)
df['Mileage'] = pd.to_numeric(df['Mileage'], errors='coerce')  # Already numeric from Part a
df['Engine'] = pd.to_numeric(df['Engine'], errors='coerce')    # Already numeric from Part a
df['Power'] = pd.to_numeric(df['Power'], errors='coerce')      # Already numeric from Part a
df['New_Price'] = pd.to_numeric(df['New_Price'], errors='coerce')  # Already numeric from Part a

# Create directory if it doesn't exist
output_dir = 'Cars_Project_Assignment-2/Data_Clean'
os.makedirs(output_dir, exist_ok=True)

# Save the updated data
df.to_csv(os.path.join(output_dir, 'data_units_removed.csv'), index=False)

# Display the first 5 rows in a clean tabular format
print(df.head().to_string(index=False))

 Unnamed: 0                             Name   Location  Year  Kilometers_Driven Fuel_Type Transmission Owner_Type  Mileage  Engine  Power  Seats  New_Price  Price
          1 Hyundai Creta 1.6 CRDi SX Option       Pune  2015              41000    Diesel       Manual      First    19.67  1582.0 126.20    5.0      11.48  12.50
          2                     Honda Jazz V    Chennai  2011              46000    Petrol       Manual      First    13.00  1199.0  88.70    5.0       8.61   4.50
          3                Maruti Ertiga VDI    Chennai  2012              87000    Diesel       Manual      First    20.77  1248.0  88.76    7.0      11.48   6.00
          4  Audi A4 New 2.0 TDI Multitronic Coimbatore  2013              40670    Diesel    Automatic     Second    15.20  1968.0 140.80    5.0      11.48  17.74
          6           Nissan Micra Diesel XV     Jaipur  2013              86999    Diesel       Manual      First    23.08  1461.0  63.10    5.0      11.48   3.50


In [15]:
#c: One-Hot Encode Categorical Variables
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import os

# Load the dataset from the previous step
df = pd.read_csv('Cars_Project_Assignment-2/Data_Clean/data_cleaned_missingValues.csv')

# Initialize OneHotEncoder (use sparse_output instead of sparse)
encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')

# Fit and transform categorical columns
categorical_cols = ['Fuel_Type', 'Transmission']
encoded_cats = encoder.fit_transform(df[categorical_cols])

# Get feature names
feature_names = encoder.get_feature_names_out(categorical_cols)

# Convert to DataFrame and merge with original
encoded_df = pd.DataFrame(encoded_cats, columns=feature_names, index=df.index)
df = pd.concat([df.drop(categorical_cols, axis=1), encoded_df], axis=1)

# Create directory if it doesn't exist
output_dir = 'Cars_Project_Assignment-2/Data_Clean'
os.makedirs(output_dir, exist_ok=True)

# Save the updated data
df.to_csv(os.path.join(output_dir, 'Categorical_data_encoded.csv'), index=False)

# Display the first 5 rows in a clean tabular format
print(df.head().to_string(index=False))

# Note (print for reference):
print("Note: 'drop='first'' avoids multicollinearity by dropping one category per variable.")

 Unnamed: 0                             Name   Location  Year  Kilometers_Driven Owner_Type  Mileage  Engine  Power  Seats  New_Price  Price  Fuel_Type_Electric  Fuel_Type_Petrol  Transmission_Manual
          1 Hyundai Creta 1.6 CRDi SX Option       Pune  2015              41000      First    19.67  1582.0 126.20    5.0      11.48  12.50                 0.0               0.0                  1.0
          2                     Honda Jazz V    Chennai  2011              46000      First    13.00  1199.0  88.70    5.0       8.61   4.50                 0.0               1.0                  1.0
          3                Maruti Ertiga VDI    Chennai  2012              87000      First    20.77  1248.0  88.76    7.0      11.48   6.00                 0.0               0.0                  1.0
          4  Audi A4 New 2.0 TDI Multitronic Coimbatore  2013              40670     Second    15.20  1968.0 140.80    5.0      11.48  17.74                 0.0               0.0                  0.0


In [17]:
#d: Create a New Feature (Car Age)
import pandas as pd
from datetime import datetime
import os

# Load the dataset from the previous step
df = pd.read_csv('Cars_Project_Assignment-2/Data_Clean/Categorical_data_encoded.csv')

# Current year
current_year = 2025

# Calculate car age
df['Car_Age'] = current_year - df['Year']

# Create directory if it doesn't exist
output_dir = 'Cars_Project_Assignment-2/Data_Clean'
os.makedirs(output_dir, exist_ok=True)

# Save the updated data
df.to_csv(os.path.join(output_dir, 'updated_data_with_current_age.csv'), index=False)

# Verify the new feature
print("First few rows with Car_Age:\n", df[['Year', 'Car_Age']].head())

First few rows with Car_Age:
    Year  Car_Age
0  2015       10
1  2011       14
2  2012       13
3  2013       12
4  2013       12


In [23]:
import pandas as pd
import os

# Load the dataset from the previous step
df = pd.read_csv('Cars_Project_Assignment-2/Data_Clean/updated_data_with_current_age.csv')

# 1. Select: Select specific columns
selected_df = df[['Name', 'Location', 'Year', 'Mileage', 'Price']]
print("Selected columns:\n", selected_df.head())

# 2. Filter: Filter cars older than 10 years
filtered_df = df[df['Car_Age'] > 10]
print("Filtered cars (age > 10 years):\n", filtered_df.head())

# 3. Rename: Rename a column
df = df.rename(columns={'Price': 'Current_Price'})
print("Renamed DataFrame (sample):\n", df[['Current_Price']].head())

# 4. Mutate: Add another derived feature (e.g., Price per Year of Age)
df['Price_Per_Year'] = df['Current_Price'] / df['Car_Age']
print("DataFrame with Price_Per_Year (sample):\n", df[['Current_Price', 'Car_Age', 'Price_Per_Year']].head())

# 5. Arrange: Sort by Price (descending)
arranged_df = df.sort_values(by='Current_Price', ascending=False)
print("Arranged by Price (top 5):\n", arranged_df[['Current_Price', 'Name']].head())

# 6. Summarize with group by: Average price by Location
summary_df = df.groupby('Location')['Current_Price'].agg(['mean', 'count', 'min', 'max']).reset_index()
print("Summary of Price by Location:\n", summary_df)

# Create directory if it doesn't exist
output_dir = 'Cars_Project_Assignment-2/results'
os.makedirs(output_dir, exist_ok=True)

# Save all Part e outputs to a single txt file
with open(os.path.join(output_dir, 'results.txt'), 'w') as f:
    f.write("--- Selected Columns ---\n")
    f.write(selected_df.head().to_string(index=False) + "\n\n")
    f.write("--- Filtered Cars (age > 10 years) ---\n")
    f.write(filtered_df.head().to_string(index=False) + "\n\n")
    f.write("--- Renamed DataFrame (Current_Price) ---\n")
    f.write(df[['Current_Price']].head().to_string(index=False) + "\n\n")
    f.write("--- Mutated DataFrame (Price_Per_Year) ---\n")
    f.write(df[['Current_Price', 'Car_Age', 'Price_Per_Year']].head().to_string(index=False) + "\n\n")
    f.write("--- Arranged by Price (top 5) ---\n")
    f.write(arranged_df[['Current_Price', 'Name']].head().to_string(index=False) + "\n\n")
    f.write("--- Summary of Price by Location ---\n")
    f.write(summary_df.to_string(index=False) + "\n")

Selected columns:
                                Name    Location  Year  Mileage  Price
0  Hyundai Creta 1.6 CRDi SX Option        Pune  2015    19.67  12.50
1                      Honda Jazz V     Chennai  2011    13.00   4.50
2                 Maruti Ertiga VDI     Chennai  2012    20.77   6.00
3   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013    15.20  17.74
4            Nissan Micra Diesel XV      Jaipur  2013    23.08   3.50
Filtered cars (age > 10 years):
    Unnamed: 0                                 Name    Location  Year  \
1           2                         Honda Jazz V     Chennai  2011   
2           3                    Maruti Ertiga VDI     Chennai  2012   
3           4      Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
4           6               Nissan Micra Diesel XV      Jaipur  2013   
6           8  Volkswagen Vento Diesel Comfortline        Pune  2013   

   Kilometers_Driven Owner_Type  Mileage  Engine   Power  Seats  New_Price  \
1              46